In [ ]:
!pip install xgboost
!pip install catboost
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 2.6 MB/s eta 0:00:00


### 데이터 준비:
- 학습 데이터셋 : train_netfilx.csv
- 검증 데이터셋(churned값 제거) : test_netflix.csv
- 검증 데이터셋(churned값만 존재) : answer_netflix.csv

In [ ]:
import pandas as pd
import numpy as np
ott = pd.read_csv('train_netflix.csv')
ott.head()

,customer_id,age,gender,subscription_type,watch_hours,last_login_days,region,device,monthly_fee,churned,payment_method,number_of_profiles,favorite_genre
0,49a5dfd9-7e69-4022-a6ad-0a1b9767fb5b,47,Other,Standard,0.70,19,Europe,Mobile,13.99,1,Gift Card,5,Sci-Fi
1,4d71f6ce-fca9-4ff7-8afa-197ac24de14b,27,Female,Standard,16.32,10,Asia,TV,13.99,0,Crypto,2,Drama
2,d3c72c38-631b-4f9e-8a0e-de103cad1a7d,53,Other,Premium,4.51,12,Oceania,TV,17.99,1,Crypto,2,Horror
3,d8079475-5be7-47e9-8782-ceb7ff61395e,58,Female,Standard,13.80,26,Oceania,Mobile,13.99,0,Debit Card,3,Action
4,8e63450a-13d6-4e83-bbb5-6aebde9152cb,48,Other,Basic,13.83,20,Asia,TV,8.99,0,Gift Card,5,Romance


In [ ]:
ott.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   customer_id         4500 non-null   object 
 1   age                 4500 non-null   int64  
 2   gender              4500 non-null   object 
 3   subscription_type   4500 non-null   object 
 4   watch_hours         4500 non-null   float64
 5   last_login_days     4500 non-null   int64  
 6   region              4500 non-null   object 
 7   device              4500 non-null   object 
 8   monthly_fee         4500 non-null   float64
 9   churned             4500 non-null   int64  
 10  payment_method      4500 non-null   object 
 11  number_of_profiles  4500 non-null   int64  
 12  favorite_genre      4500 non-null   object 
dtypes: float64(2), int64(4), object(7)
memory usage: 457.2+ KB


### 데이터 정제
|변수명|타입|설명|특이사항|독립/종속 변수|
|-----|-----|-----|-----|----|
| customer_id|고유값|식별자|모델 학습에서 제외 |독립|
| age| 수치형 범주 | 나이 | 범주형 처리  | 독립|
| gender  | 범주형    | 성별   | One-hot or Label or target|독립|
| subscription_type | 범주형  | 베이직/프리미엄 등  | monthly_fee와 중복됨 | 독립|
| watch\_hours | 수치형 범주 | 시청 시간 구간형 (예: 0-5, 5-10시간) | 인코딩 가능 | 독립|
| last_login_days | 수치형 범주 | 마지막 로그인 이후 일수(최대 60일) | 범주형 처리 |독립|
| region| 범주형| 대륙(Asia, Europe 등) | 범주형 처리  |독립|
| device | 범주형 | 모바일, PC 등 | 범주형 처리  |독립|
| monthly_fee | 수치형 범주 | 금액 구간  | subscription_type과 중복됨 |독립|
| payment_method | 범주형  | 카드, 계좌이체 등  | 범주형 처리 |독립|
| number_of_profiles |수치형 범주 | 예: 1~5개 | 그대로 사용 또는 구간화 |독립|
| favorite\_genre | 범주형 | 최대 7개 장르 | 범주형 처리  |독립|
| churned  | 이진형    | 타깃 변수 (0=잔류, 1=이탈)  | -  |종속|


#### churned 값을 제외하고 모든값이 독립인것을 확인

#### customer_id, subscription_type 제거
 - customer_id: 고유값으로 학습에서 제외 대상이기 때문에 드롭
 - subscription_type - 구독 유형 (베이직,프리미엄,스탠다드)
 - monthly_fee - 구독 금액
 - ssubscription_type과 monthly_fee는 다중공선성 이슈로 둘 중 하나를 제거하는 것이 바람직합니다.

 - 이번 분석에서는 subscription_type을 제거하고 monthly_fee만 남겨 진행하는 것이 합리적이라 생각되어 subscription_type를 제거 후 진행.

In [ ]:
ott = ott.drop(['customer_id', 'subscription_type'], axis=1)

### 모델 선택 : Random Forest, XGBoost, Catboost
| 모델           | 선정 이유 요약                      |
| ------------ | ----------------------------- |
| RandomForest | 다양한 변수 간 상호작용 포착, 과적합 방지, 안정성 해석력, 빠른 벤치마크             |
| CatBoost     | 범주형 자동처리, 과적합 억제, 실무 친화성      |
| XGBoost      | 불균형 데이터 대응력, 빠른 연산, 병렬 처리, 성능 최적화, 대규모 데이터 효율성, 정밀 튜닝 가능 |

RandomForest

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

features = ['age', 'watch_hours', 'monthly_fee', 'number_of_profiles', 'last_login_days']
target = 'churned'

X = ott[features]
y = ott[target]


# train, test 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# XGBoost model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = rf_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print('Prediction Accuracy:', acc)

Prediction Accuracy: 0.9368888888888889


Catboost

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from catboost import CatBoostClassifier

features = ['age', 'watch_hours', 'monthly_fee', 'number_of_profiles', 'last_login_days']
target = 'churned'

X = ott[features]
y = ott[target]


# train, test 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# XGBoost model
cb_model = CatBoostClassifier(verbose=False)
cb_model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = cb_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print('Prediction Accuracy:', acc)

Prediction Accuracy: 0.944


XGBoost

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

features = ['age', 'watch_hours', 'monthly_fee', 'number_of_profiles', 'last_login_days']
target = 'churned'

X = ott[features]
y = ott[target]


# train, test 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# XGBoost model
xg_model = XGBClassifier()
xg_model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = xg_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print('Prediction Accuracy:', acc)

Prediction Accuracy: 0.9422222222222222


#### 모델 학습 결과: RF, CB, XGB 중 CatBoost가 가장 높은 성능을 보여준다.
1. RandomForest
 * Accuracy: 0.936
2. Catboost
 * Accuracy: 0.944
3. XGBoost
 * Accuracy: 0.942

### 인코딩


- customer_id를 제거함으로 차원 폭발이 일어날만한 요소가 없기 때문에 label encoding이 필요해 보이지 않음.
- target encoding과 one-hot encoding만 진행
 1. case 1: only target encoding
 2. case 2: target encoding, one-hot encoding
 3. case 3: only one-hot encoding

#### case1:

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from category_encoders import TargetEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 1. 피처 및 타겟 설정
features = ['age', 'watch_hours', 'monthly_fee', 'number_of_profiles', 'last_login_days',
            'gender', 'region', 'device', 'payment_method', 'favorite_genre']
target = 'churned'

X = ott[features].copy()
y = ott[target].copy()

# 2. 전체 범주형 변수 목록
categorical_cols = ['gender', 'region', 'device', 'payment_method', 'favorite_genre']

# 3. train-test 분할 (타겟 누출 방지)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Target Encoding 적용
target_encoder = TargetEncoder(cols=categorical_cols)
X_train = target_encoder.fit_transform(X_train, y_train)
X_test = target_encoder.transform(X_test)

# 5. XGBoost 모델 학습
model = XGBClassifier(random_state=42)
model.fit(X_train, y_train)

# 6. 예측
y_pred = model.predict(X_test)

# 7. 성능 평가
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# 5. 결과 출력
print("Case : 모든 범주형 특성 Target 인코딩 적용 XGBoost 모델 성능")
print(f"결정계수 (R²): {r2:.4f}")
print(f"평균 절대 오차 (MAE): {mae:.4f}")
print(f"평균 제곱 오차 (MSE): {mse:.4f}")
print(f"평균 제곱근 오차 (RMSE): {rmse:.4f}")


Case : 모든 범주형 특성 Target 인코딩 적용 XGBoost 모델 성능
결정계수 (R²): 0.9644
평균 절대 오차 (MAE): 0.0089
평균 제곱 오차 (MSE): 0.0089
평균 제곱근 오차 (RMSE): 0.0943


#### case2:

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from category_encoders import TargetEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 1. 입력 특성과 타겟 설정
features = ['age', 'watch_hours', 'monthly_fee', 'number_of_profiles', 'last_login_days',
            'gender', 'region', 'device', 'payment_method', 'favorite_genre']
target = 'churned'

X = ott[features].copy()
y = ott[target].copy()

# 2. gender만 One-Hot 인코딩
X = pd.get_dummies(X, columns=['gender'], drop_first=True)

# 3. train-test 분할 (분할 후 Target Encoding)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Target Encoding (나머지 범주형 변수들)
te_features = ['region', 'device', 'payment_method', 'favorite_genre']
target_encoder = TargetEncoder(cols=te_features)

X_train = target_encoder.fit_transform(X_train, y_train)
X_test = target_encoder.transform(X_test)

# 5. 모델 훈련
model = XGBClassifier(random_state=42)
model.fit(X_train, y_train)

# 6. 예측
y_pred = model.predict(X_test)

# 7. 평가
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# 7. 결과 출력
print("Case 2: 모든 범주형 특성에 Target, One-Hot 인코딩 XGBoost 성능")
print(f"결정계수 (R²): {r2:.4f}")
print(f"평균 절대 오차 (MAE): {mae:.4f}")
print(f"평균 제곱 오차 (MSE): {mse:.4f}")
print(f"평균 제곱근 오차 (RMSE): {rmse:.4f}")

Case 2: 모든 범주형 특성에 Target, One-Hot 인코딩 XGBoost 성능
결정계수 (R²): 0.9556
평균 절대 오차 (MAE): 0.0111
평균 제곱 오차 (MSE): 0.0111
평균 제곱근 오차 (RMSE): 0.1054


#### case3:

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd

# Define features and target
features = ['age', 'watch_hours', 'monthly_fee', 'number_of_profiles', 'last_login_days','gender','region', 'device', 'payment_method', 'favorite_genre']
target = 'churned'

# Separate features and target
X = ott[features]
y = ott[target]

# 1. 데이터 복사 및 범주형 인코딩
X_case3 = pd.get_dummies(X.copy(), drop_first=True)

# 2. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_case3, y, test_size=0.2, random_state=42)

# 3. XGBoost 모델 학습 및 예측
model = XGBClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# 4. 회귀 성능 평가 지표 계산
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# 5. 결과 출력
print("Case 3: 모든 범주형 특성에 One-hot 인코딩 적용 XGBoost 모델 성능")
print(f"결정계수 (R²): {r2:.4f}")
print(f"평균 절대 오차 (MAE): {mae:.4f}")
print(f"평균 제곱 오차 (MSE): {mse:.4f}")
print(f"평균 제곱근 오차 (RMSE): {rmse:.4f}")

Case 3: 모든 범주형 특성에 One-hot 인코딩 적용 XGBoost 모델 성능
결정계수 (R²): 0.9467
평균 절대 오차 (MAE): 0.0133
평균 제곱 오차 (MSE): 0.0133
평균 제곱근 오차 (RMSE): 0.1155


- 모델 학습: 각 각 다른인코딩을 적용하였고, 최종적으로 XGBoost가 가장 높게 출력되어 XGBoost 모델을 선정하였고, 인코딩은 case1인 taget 인코딩만 사용한 것이 R²와 오차가 가장 적게 출력됨.
 * 결정계수 (R²): 0.9644
 * 평균 절대 오차 (MAE): 0.0089
 * 평균 제곱 오차 (MSE): 0.0089
 * 평균 제곱근 오차 (RMSE): 0.0943

### 하이퍼 파라미터 튜닝

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

# Assuming X_train and y_train from the preferred case (Case 1) are available
# We also need X_test and y_test to evaluate the tuned model

# Define the parameter grid for XGBoost
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=XGBClassifier(random_state=42),
                           param_grid=param_grid,
                           scoring='r2',  # Using R^2 as per previous evaluations
                           cv=3,          # Cross-validation folds
                           verbose=1,
                           n_jobs=-1)     # Use all available cores

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print("Best hyperparameters found:")
print(grid_search.best_params_)
print(f"Best R^2 score: {grid_search.best_score_:.4f}")

# Train the model with the best parameters
best_xgb_model = XGBClassifier(**grid_search.best_params_, random_state=42)
best_xgb_model.fit(X_train, y_train)

# Evaluate the best model on the test set
y_pred_tuned = best_xgb_model.predict(X_test)

r2_tuned = r2_score(y_test, y_pred_tuned)
mae_tuned = mean_absolute_error(y_test, y_pred_tuned)
mse_tuned = mean_squared_error(y_test, y_pred_tuned)
rmse_tuned = np.sqrt(mse_tuned)

print("\n Tuned XGBoost 모델 성능:")
print(f"결정계수 (R²): {r2_tuned:.4f}")
print(f"평균 절대 오차 (MAE): {mae_tuned:.4f}")
print(f"평균 제곱 오차 (MSE): {mse_tuned:.4f}")
print(f"평균 제곱근 오차 (RMSE): {rmse_tuned:.4f}")

Fitting 3 folds for each of 243 candidates, totalling 729 fits
Best hyperparameters found:
{'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200, 'subsample': 1.0}
Best R^2 score: 0.9422

📊 Tuned XGBoost 모델 성능:
결정계수 (R²): 0.9600
평균 절대 오차 (MAE): 0.0100
평균 제곱 오차 (MSE): 0.0100
평균 제곱근 오차 (RMSE): 0.1000


#### 튜닝 결과
- 튜닝 이전
  * 결정계수 (R²): 0.9644
  * 평균 절대 오차 (MAE): 0.0089
  * 평균 제곱 오차 (MSE): 0.0089
  * 평균 제곱근 오차 (RMSE): 0.0943
- 튜닝 이후
  * 결정계수 (R²): 0.9600
  * 평균 절대 오차 (MAE): 0.0100
  * 평균 제곱 오차 (MSE): 0.0100
  * 평균 제곱근 오차 (RMSE): 0.1000
- 하이퍼 파라미터 튜닝 이후 오히려 R²값이 떨어지고, 오차값이 커짐.
- 따라서 튜닝을 하지않은 이전 모델을 최종 모델로 선택

### 피클 파일 저장

In [ ]:
import pickle

# Assuming the trained model object (e.g., 'model' or 'best_xgb_model') is available in memory
# Based on previous steps, 'model' from cell 4AfEuQ4bwcqX or bf7e45e4 (Case 1 training) is the likely final model.

try:
    # Save the trained model
    with open('xgb_churn_model.pkl', 'wb') as f:
        pickle.dump(model, f) # Assuming 'model' holds the final trained model

    print("Trained model saved as xgb_churn_model.pkl")

except NameError:
    print("Error: Trained model object not found in memory. Please ensure the model training cell was run successfully.")
except Exception as e:
    print(f"An unexpected error occurred during model saving: {e}")

Trained model saved as xgb_churn_model.pkl


In [ ]:
import pandas as pd
from category_encoders import TargetEncoder
from sklearn.model_selection import train_test_split
import pickle

features = ['age', 'watch_hours', 'monthly_fee', 'number_of_profiles', 'last_login_days', 'gender', 'region', 'device', 'payment_method', 'favorite_genre']
target = 'churned'

X = ott[features]
y = ott[target]


categorical_features_te = ['gender', 'region', 'device', 'payment_method', 'favorite_genre']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


target_encoder_case1 = TargetEncoder(cols=categorical_features_te)
X_train_encoded_case1 = target_encoder_case1.fit_transform(X_train, y_train)


X_test_encoded_case1 = target_encoder_case1.transform(X_test)

with open('target_encoder_case1.pkl', 'wb') as f:
    pickle.dump(target_encoder_case1, f)

print("Case 1 Encoding Applied and Encoder Saved.")
display(X_train_encoded_case1.head())

Case 1 Encoding Applied and Encoder Saved.


,age,watch_hours,monthly_fee,number_of_profiles,last_login_days,gender,region,device,payment_method,favorite_genre
4137,38,2.70,8.99,1,49,0.513688,0.493333,0.512535,0.441576,0.506823
3525,39,9.89,8.99,4,45,0.495359,0.479930,0.512535,0.577035,0.506823
2652,33,11.36,8.99,1,51,0.513688,0.525237,0.481534,0.462988,0.515326
605,36,22.10,17.99,4,40,0.513688,0.516026,0.512465,0.588798,0.506823
2194,68,19.91,17.99,3,54,0.501279,0.525237,0.512535,0.577035,0.509690


### 모델 성능 평가 (새로운 데이터 셋으로 비교)

In [ ]:
import pandas as pd
import pickle
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np

# Load the test data
test_ott = pd.read_csv('test_netflix.csv')

# Load the saved encoder and model
with open('target_encoder_case1.pkl', 'rb') as f:
    loaded_target_encoder = pickle.load(f)

with open('xgb_churn_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

# Define the features used during training
features = ['age', 'watch_hours', 'monthly_fee', 'number_of_profiles', 'last_login_days',
            'gender', 'region', 'device', 'payment_method', 'favorite_genre']

# Select the features from the test data
X_test_new = test_ott[features].copy()

# Apply the loaded target encoder to the test data
X_test_new_encoded = loaded_target_encoder.transform(X_test_new)

# Make predictions on the encoded test data
test_predictions = loaded_model.predict(X_test_new_encoded)

# Load the actual churn values
answer_data = pd.read_csv('answer_netflix.csv')
actual_churned = answer_data['churned']

# Evaluate the model on the test data
accuracy = accuracy_score(actual_churned, test_predictions)
conf_matrix = confusion_matrix(actual_churned, test_predictions)

print(f"예측 결과와 실제 정답 간 일치율 (정확도): {accuracy:.4f}")

# Create a DataFrame to compare actual vs. predicted
comparison_df = pd.DataFrame({'Actual Churn': actual_churned, 'Predicted Churn': test_predictions})
display(comparison_df)

예측 결과와 실제 정답 간 일치율 (정확도): 0.9800


,Actual Churn,Predicted Churn
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
495,1,1
496,1,1
497,1,1
498,1,1


### 예측 결과

초기 모델 학습 단계에서 다양한 인코딩 방식과 모델을 비교한 결과, **Case 1 모든 범주형 변수는 Target Encoding )**을 적용한 **XGBoost 모델**이 가장 우수한 성능을 보였습니다.

하이퍼파라미터 튜닝을 시도했으나 성능 개선이 이루어지지 않아 초기 Case 1 모델을 최종 모델로 선정하였습니다.

최종 모델의 성능 지표는 다음과 같습니다:

*   **결정계수 (R²):** 0.9644
*   **평균 절대 오차 (MAE):** 0.0089
*   **평균 제곱 오차 (MSE):** 0.0089
*   **평균 제곱근 오차 (RMSE):** 0.0943

이러한 지표는 모델이 고객 이탈 여부를 상당히 정확하게 예측함을 나타냅니다. R² 값이 0.9644로 매우 높아 모델이 데이터의 변동성을 잘 설명하고 있음을 알 수 있습니다. 또한, MAE, MSE, RMSE 값이 낮아 예측 오류가 적음을 확인할 수 있습니다.

초기에 분리했던 데이터(test_netfilx.csv(churned값 제거))와 해당 파일의 churned값이 있는 answer_netflix.csv파일을 로드하여 실제 적용 및 예측 값 확인

예측 결과와 실제 정답 간 일치율 (정확도): 0.9800 로 기존 지표보다 높은 성능을 보여줍니다.